<a href="https://www.kaggle.com/code/averma111/pytorch-widedeep-pss3e18?scriptVersionId=134979275" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
%%capture
!pip install pytorch-widedeep

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os


import torch
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.training import Trainer
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy, Precision,F1Score,Recall

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import itertools

In [ ]:
class Datapreparation(object):
    
    def __init__(self,root_path):
        self.root_path = root_path
        
    def get_dataframe(self,filename):
        return pd.read_csv(os.path.join(self.root_path,filename))
    
    def summary(self,text, df):
        summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
        summary['null'] = df.isnull().sum()
        summary['unique'] = df.nunique()
        summary['min'] = df.min()
        summary['median'] = df.median()
        summary['max'] = df.max()
        summary['mean'] = df.mean()
        summary['std'] = df.std()
        summary['duplicate'] = df.duplicated().sum()
        return summary
    
    

    
data = Datapreparation('/kaggle/input/playground-series-s3e18')
train=data.get_dataframe('train.csv')


In [ ]:
data.summary('train',train)

In [ ]:
# Categorical columns
cat_embed_cols = ['fr_COO','fr_COO2','NumHeteroatoms']
# Continous columns
continuous_cols = ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
       'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
       'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
       'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
        'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9']

# TARGET
target_col = ['EC1','EC2']
target = train[target_col].values

In [ ]:
# deeptabular
tab_preprocessor = TabPreprocessor(
    embed_cols=cat_embed_cols, continuous_cols=continuous_cols
)
X_tab = tab_preprocessor.fit_transform(train)
tab_preprocessor.cat_embed_input

In [ ]:
# Model 
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    cat_embed_dropout=0.1,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[8, 4],
    mlp_dropout=0.5,
    mlp_activation="leaky_relu"
)

tab_model = WideDeep(deeptabular=tab_mlp,pred_dim=2)
tab_model

In [ ]:
tab_trainer = Trainer(
    model=tab_model,
    objective="multiclass",
    optimizers=torch.optim.Adam(tab_model.parameters(), lr=0.001),
    metrics=[Accuracy, Precision,F1Score,Recall],
)

In [ ]:
tab_trainer.fit(X_tab=X_tab, target=target, n_epochs=10, batch_size=32, val_split=0.2)

In [ ]:
test=data.get_dataframe('test.csv')
test = data.rename_column(test)
test.head()

In [ ]:
#X_wide_te = wide_preprocessor.transform(test_updated)
X_tab_te = tab_preprocessor.transform(test_updated)
preds = tab_trainer.predict_proba(X_tab=X_tab_te)
print(f'The  probabilities are {preds}')

In [ ]:
class Submit:
    
    def submit_predictions(self,test_updated):
        df_submit = pd.DataFrame(data={'id': test_updated['id'],'Machine Failure':prediction_1})
        df_submit.to_csv('submission.csv',index=False)
        print('Submission Completed!!')
        return df_submit
        
        
submit = Submit()
df_submit=submit.submit_predictions(test_updated)